# Azure chat completions example (preview)
In this example we'll try to go over all operations needed to get chat completions working using the Azure endpoints. \
This example focuses on chat completions but also touches on some other operations that are also available using the API. This example is meant to be a quick way of showing simple operations and is not meant as a tutorial.

In [ ]:
import openai

## Setup
For the following sections to work properly we first have to setup some things. Let's start with the `api_base` and `api_version`. To find your `api_base` go to https://portal.azure.com, find your resource and then under "Resource Management" -> "Keys and Endpoints" look for the "Endpoint" value.

In [ ]:
openai.api_version = '2023-03-15-preview'
openai.api_base = '' # Please add your endpoint here

We next have to setup the `api_type` and `api_key`. We can either get the key from the portal or we can get it through Microsoft Active Directory Authentication. Depending on this the `api_type` is either `azure` or `azure_ad`.

### Setup: Portal
Let's first look at getting the key from the portal. Go to https://portal.azure.com, find your resource and then under "Resource Management" -> "Keys and Endpoints" look for one of the "Keys" values.

In [ ]:
openai.api_type = 'azure'
openai.api_key = ''  # Please add your api key here

### (Optional) Setup: Microsoft Active Directory Authentication
Let's now see how we can get a key via Microsoft Active Directory Authentication. Uncomment the following code if you want to use Active Directory Authentication instead of keys from the portal.

In [ ]:
# from azure.identity import DefaultAzureCredential

# default_credential = DefaultAzureCredential()
# token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

# openai.api_type = 'azure_ad'
# openai.api_key = token.token

## Deployments
In this section we are going to create a deployment using the `gpt-35-turbo` model that we can then use to create chat completions.

### Deployments: Create manually
Let's create a deployment using the `gpt-35-turbo` model. Go to https://portal.azure.com, find your resource and then under "Resource Management" -> "Model deployments" create a new `gpt-35-turbo` deployment. 

In [ ]:
deployment_id = "" # Fill in the deployment id from the portal here

### (Optional) Deployments: Create programatically
We can also create the deployment using code. Note that you can only create one deployment per model.

In [ ]:
model = "gpt-35-turbo"

# Now let's create the deployment
print(f'Creating a new deployment with model: {model}')
result = openai.Deployment.create(model=model, scale_settings={"scale_type":"standard"})
deployment_id = result["id"]
print(f'Successfully created deployment with id: {deployment_id}')

### (Optional) Deployments: Wait for deployment to succeed
Now let's check the status of the newly created deployment and wait till it is succeeded.

In [ ]:
print(f'Checking for deployment status.')
resp = openai.Deployment.retrieve(id=deployment_id)
status = resp["status"]
print(f'Deployment {deployment_id} has status: {status}')
while status not in ["succeeded", "failed"]:
    resp = openai.Deployment.retrieve(id=deployment_id)
    status = resp["status"]
    print(f'Deployment {deployment_id} has status: {status}')

### Create chat completion
Now let's send a sample chat completion to the deployment.

In [ ]:
# For all possible arguments see https://platform.openai.com/docs/api-reference/chat-completions/create
response = openai.ChatCompletion.create(
    deployment_id=deployment_id,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
)

print(f"{response.choices[0].message.role}: {response.choices[0].message.content}")

We can also stream the response.


In [ ]:
response = openai.ChatCompletion.create(
    deployment_id=deployment_id,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
    stream=True
)

for chunk in response:
    delta = chunk.choices[0].delta

    if "role" in delta.keys():
        print(delta.role + ": ", end="", flush=True)
    if "content" in delta.keys():
        print(delta.content, end="", flush=True)

### (Optional) Deployments: Delete
Finally let's delete the deployment.

In [ ]:
print(f'Deleting deployment: {deployment_id}')
openai.Deployment.delete(sid=deployment_id)